In [7]:
!pip3 install pandas pyarrow fastparquet transformers tensorflow scikit-learn

  Using cached scikit_learn-1.3.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.4/165.4 kB 4.7 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
Using cached scikit_learn-1.3.2-cp311-cp311-macosx_12_0_arm64.whl (9.4 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 25.7 MB/s eta 0:00:0000:0100:01
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)


In [2]:
import pandas as pd

# viewing the data
top_data = "data/top_level-00000-of-00001.parquet"
top_df = pd.read_parquet(top_data)
top_df

,desc,decl,bodies
0,'load a Python source file and compile it to b...,def _load_file(filename):\n,"fp = open(filename, 'rb')\n source = (f..."
1,'load a module without importing it\n_load_mod...,def _load_module(filename):\n,"import magics, marshal\n fp = open(file..."
2,'diassembles a given code block \'co\''\n,"def uncompyle(version, co, out=None, showasm=0...",assert (type(co) == types.CodeType)\n _...
3,'decompile Python byte-code file (.pyc)'\n,"def uncompyle_file(filename, outstream=None, s...","(version, co) = _load_module(filename)\n ..."
4,'in_base base directory for input files\no...,"def main(in_base, out_base, files, codes, outf...",def _get_outstream(outfile):\n dir ...
...,...,...,...
148597,'Start a server instance. This method blocks u...,"def run(app=None, server='wsgiref', host='127....",if NORUN:\n return\n if (reloade...
148598,'Get a rendered template as a string iterator....,"def template(*args, **kwargs):\n",tpl = (args[0] if args else None)\n ada...
148599,'Decorator: renders a template for a handler.\...,"def view(tpl_name, **defaults):\n",def decorator(func):\n @functools.w...
148600,'Return a string describing the probable encod...,"def description_of(lines, name=u'stdin'):\n",u = UniversalDetector()\n for line in l...


In [3]:
from transformers import BertTokenizer, TFBertModel, T5Tokenizer, TFT5Model

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")
t5_model = TFT5Model.from_pretrained("t5-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
input_text = "Hello, my dog is not cute, no really"
bert_tokenized_sentence = bert_tokenizer(
    input_text, max_length=15, truncation=True, return_tensors="tf"
)

for key in bert_tokenized_sentence:
    print(key, ": ", bert_tokenized_sentence[key])
print(bert_tokenizer.decode(bert_tokenized_sentence["input_ids"][0]))

input_ids :  tf.Tensor([[  101  7592  1010  2026  3899  2003  2025 10140  1010  2053  2428   102]], shape=(1, 12), dtype=int32)
token_type_ids :  tf.Tensor([[0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 12), dtype=int32)
attention_mask :  tf.Tensor([[1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 12), dtype=int32)
[CLS] hello, my dog is not cute, no really [SEP]


In [8]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(top_df, test_size=0.2)